In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

### 1. Data processing in BigQuery
1. Get one week of data from this public available BiqQuery table: gdelt-bq.gdeltv2.geg_gcnlapi
2. Create hosts table, articles table and entities table, with node ids assigned
3. Create 2 edge index tables: host to article and article to entity


### 2. Read pre-processed data from BigQuery into Dataframes
Hosts table, article table, entities table, and edge indices between them.

In [2]:
# BigQuery api client
bq = bigquery.Client(location="US")


In [3]:
def nodes_tables_from_bq(bq):
    host_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.hosts_table` 
        """
    article_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.articles_table` 
        """
    entity_query = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.entities_table` 
        """
    query_job = bq.query(host_query, location="US") 
    hosts = query_job.to_dataframe() 
    query_job = bq.query(article_query, location="US") 
    articles = query_job.to_dataframe() 
    query_job = bq.query(entity_query, location="US") 
    entities = query_job.to_dataframe() 
    return hosts, articles, entities

In [4]:
def load_nodes_tables(bq):
    exists_hosts = os.path.isfile('data/hosts_table.csv')
    exists_articles = os.path.isfile('data/articles_table.csv')
    exists_entities = os.path.isfile('data/entities_table.csv')
    if exists_hosts and exists_articles and exists_entities:
        print("Read from local csv files.")
        hosts = pd.read_csv('data/hosts_table.csv')
        articles = pd.read_csv('data/articles_table.csv')
        entities = pd.read_csv('data/entities_table.csv')
    else:
        print("Read from BigQuery.")
        hosts, articles, entities = nodes_tables_from_bq(bq)
        hosts.to_csv('data/hosts_table.csv')
        articles.to_csv('data/articles_table.csv')
        entities.to_csv('data/entities_table.csv')
    
    return hosts, articles, entities

In [5]:
# load nodes tables into dataframes
hosts_table, articles_table, entities_table = load_nodes_tables(bq)
print(hosts_table.shape, articles_table.shape, entities_table.shape)


Read from BigQuery.


In [7]:
print(hosts_table.shape, articles_table.shape, entities_table.shape)

(20033, 2) (1048852, 6) (1289112, 3)


In [26]:
# sort the dataframes by node ids
hosts_table = hosts_table.sort_values(by=['host_node_id'], ascending=True).reset_index(drop=True)
articles_table = articles_table.sort_values(by=['article_node_id'], ascending=True).reset_index(drop=True)
entities_table = entities_table.sort_values(by=['entity_node_id'], ascending=True).reset_index(drop=True)


In [28]:
display(hosts_table.head(5))
display(articles_table.head(5))
display(entities_table.head(5))


,host_node_id,host
0,0,www.berchtesgadener-anzeiger.de
1,1,www.toponline.ch
2,2,news.yahoo.com
3,3,abcnews.go.com
4,4,www.dailyliberal.com.au


,article_node_id,host,url,lang,magnitude,score
0,0,www.workersliberty.org,https://www.workersliberty.org/index.php/audio,en,15.9,0.0
1,1,www.wkyc.com,https://www.wkyc.com/article/news/nation-world...,en,24.6,-0.4
2,2,townhall.com,https://townhall.com/tipsheet/mattvespa/2023/1...,en,13.2,-0.4
3,3,www.crowdfundinsider.com,https://www.crowdfundinsider.com/2023/10/21440...,en,5.9,0.2
4,4,www.ktep.org,https://www.ktep.org/u-s-news/2023-10-17/once-...,en,5.2,0.0


,entity_node_id,mid,type
0,0,/m/0138vk,LOCATION
1,1,/m/02phmc9,LOCATION
2,2,/m/04gsnk1,ORGANIZATION
3,3,/m/02d1f0,PERSON
4,4,/m/011hrk,LOCATION


In [13]:
# use random node feature for host, because hosts do not have rich features
random_feature_dim = 32
host_features = torch.randn(hosts_table.shape[0], random_feature_dim)  # Random features for each node
host_features.shape

torch.Size([20033, 32])

In [ ]:
# features for article: lang, magnitude, score
unique_langs, article_langs = np.unique(geg['lang'].values, return_inverse=True)
article_features = pd.get_dummies(article_langs)
article_features['magniture'] = geg['magnitude']
article_features['score'] = geg['score']
article_features = torch.from_numpy(article_features.values).to(torch.float)

In [ ]:
def read_geg_from_bq(dataset='week'):
    bq = bigquery.Client(location="US")
    dataset_1_month = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.geg_with_mid` 
        WHERE date between TIMESTAMP("2023-11-01") and TIMESTAMP("2023-11-30")
        """
    dataset_1_week = """  
        SELECT * 
        FROM `gannett-datarevenue.zz_test_pang.geg_with_mid` 
        WHERE date between TIMESTAMP("2023-11-19") and TIMESTAMP("2023-11-25")
        """
    
    query = dataset_1_month if dataset == 'month' else dataset_1_week

    query_job = bq.query(query, location="US") 
    df = query_job.to_dataframe() 
    return df
    

In [ ]:
def load_one_week_data():
    df = None
    exists = os.path.isfile('data/geg_1_week.csv')
    if exists:
        print("read from local file data/geg_1_week.csv")
        df = pd.read_csv('data/geg_1_week.csv')
    else:
        print("read from BigQuery")
        df = read_geg_from_bq('week')
        df.to_csv('data/geg_1_week.csv')
    
    return df
    

In [ ]:
geg = load_one_week_data()

print(geg.shape)

In [ ]:
geg.sample(5)

### Node features:
1. Node Type - News Outlet (identified by host url), Article (identified by url), Entity (identified by mid)
2. Node Subtype - "News Outlet" for News Outlet nodes, "Article" for Article nodes, and value of mid for Entity nodes.  
3. Magnitude - 0 for news outlets and entities, value of magnitude for articles
4. Score - 0 for news outlets and entities, value of score for articles


In [ ]:
# Create a mapping from unique news outlet (host) to range (0, num_news_outlets):
unique_news_outlets = geg['host'].unique()
unique_news_outlets = pd.DataFrame(data={
    'host': unique_news_outlets,
    'node_id': pd.RangeIndex(len(unique_news_outlets)),
})

print("Mapping of hosts to node ids:")
print("==========================================")
print(unique_news_outlets.head())
print()

In [ ]:
# Assign unique node ids to hosts
unique_hosts, host_node_ids = np.unique(geg['host'].values, return_inverse=True)
geg['host_node_id'] = host_node_ids


In [ ]:
# Assign unique node ids to articles, which are identified by urls
unique_articles, article_node_ids = np.unique(geg['url'].values, return_inverse=True)
geg['article_node_id'] = article_node_ids

In [ ]:
# Assign unique node ids to entities, which are identified by mids
unique_entities, entity_node_ids = np.unique(geg['mid'].values, return_inverse=True)
geg['entity_node_id'] = entity_node_ids

In [ ]:
print(len(unique_hosts), len(unique_articles), len(unique_entities))

unique_hosts

In [ ]:
geg.head()

In [ ]:
# features for article: lang, magnitude, score

# 

unique_langs, article_langs = np.unique(geg['lang'].values, return_inverse=True)
article_features = pd.get_dummies(article_langs)
article_features['magniture'] = geg['magnitude']
article_features['score'] = geg['score']
article_features = torch.from_numpy(article_features.values).to(torch.float)


In [ ]:
# features for entity: type
unique_entity_types, entity_types = np.unique(geg['type'].values, return_inverse=True)
entity_features = pd.get_dummies(entity_types)
entity_features = torch.from_numpy(entity_features.values).to(torch.float)

In [ ]:
data = HeteroData()
# Save node indices:
data["host"].node_id = torch.arange(len(unique_hosts))
data["article"].node_id = torch.arange(len(unique_articles))
data["entity"].node_id = torch.arange(len(unique_entities))
# Add the node features and edge indices:
data["host"].x = movie_feat
data

In [ ]:
host_features.shape

In [ ]:
geg.shape

In [ ]:
usa_today = geg[geg['host'].str.contains('usatoday')]


In [ ]:
usa_today.shape

In [ ]:
nytimes = geg[geg['host'].str.contains('nytimes')]

In [ ]:
nytimes.shape